In [5]:
# TargetEncoder
#!pip install -q scikit-learn==1.4
# KerasClassifier
#!pip install -q --no-deps scikeras

In [6]:
#현재 커널에 설치가 안되어서 삽질..
#import sys
#import subprocess
# 현재 Python 실행 파일 경로를 사용하여 pip 설치 명령 실행
#subprocess.check_call([sys.executable, "-m", "pip", "install", "scikeras"])


In [1]:
import pandas as pd
import numpy as np
import keras
from tensorflow.keras.callbacks import LearningRateScheduler, EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, FunctionTransformer
from sklearn.preprocessing import OneHotEncoder, TargetEncoder
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, roc_auc_score
from scikeras.wrappers import KerasClassifier
from sklearn.base import clone

2024-07-24 12:46:49.341570: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, confusion_matrix, classification_report, roc_curve, auc
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from tqdm import tqdm

In [3]:
# Read and split the data
train_origin = pd.read_csv('/Users/jaesolshin/내 드라이브/2024-2/Google ML Bootcamp2024/data/playground1/train.csv')
train = train_origin.sample(frac=0.01, random_state = 42)

In [10]:
# 범주형 변수를 팩터로 변환 (카테고리형)
train.iloc[:,[1,3,4,5,6,7,9]] = train.iloc[:,[1,3,4,5,6,7,9]].astype('category')

# 최소-최대 정규화 (Min-Max 스케일링)
scaler = MinMaxScaler()
train.iloc[:,[2,8,10]] = scaler.fit_transform(train.iloc[:,[2,8,10]])

# 이분변수 생성: "Annual_Premium" == 2630.0 인 경우
train['Annual_Premium_Binary'] = (train['Annual_Premium'] == 2630.0).astype('category')

# 로그 변환된 "Annual_Premium" 변수 생성
train['Annual_Premium_Log'] = np.where(train['Annual_Premium'] > 0, np.log1p(train['Annual_Premium']), 0)

# 예측에 필요 없는 'id'와 'Annual_Premium' 변수를 드롭
train = train.drop(columns=['id', 'Annual_Premium'])

# 원-핫 인코딩 (One-Hot Encoding)
train = pd.get_dummies(train, columns=['Gender', 'Driving_License', 'Region_Code', 'Previously_Insured', 'Vehicle_Age', 'Vehicle_Damage', 'Policy_Sales_Channel', 'Annual_Premium_Binary'])

# XGBoost에서 발생하는 문제 해결
train.columns = train.columns.str.replace('[', '').str.replace(']', '').str.replace('<', '')

In [39]:
from sklearn.model_selection import StratifiedKFold
from tqdm import tqdm
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasClassifier
from sklearn.metrics import accuracy_score, roc_auc_score

# 평가 함수 정의
def evaluate_model(y_true, y_pred, y_proba):
    return {
        'Accuracy': accuracy_score(y_true, y_pred),
        'ROC AUC': roc_auc_score(y_true, y_proba)
    }

# 모델을 초기화하는 함수 정의
# The Keras model with two hidden layers
def make_model(meta):
    model = keras.models.Sequential()
    model.add(keras.layers.Input(meta["X_shape_"][1:]))
    model.add(keras.layers.Dense(128, kernel_initializer='lecun_normal', activation='selu'))
    model.add(keras.layers.Dense(64, kernel_initializer='lecun_normal', activation='selu'))
    model.add(keras.layers.Dense(1, kernel_initializer='lecun_normal', activation='sigmoid'))
    return model


# 교차 검증 설정
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
X = train.drop(columns=['Response'])
y = train['Response']

# 결과를 저장할 리스트 초기화
results = []
probas = []

# 모델 학습 및 평가
for fold, (train_index, valid_index) in enumerate(tqdm(skf.split(X, y), desc="Training model")):
    X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]

    # 모델 초기화 (새로운 인스턴스를 생성)
    model_instance = KerasClassifier(
    get_model,
    loss="binary_crossentropy",
    optimizer=keras.optimizers.AdamW(learning_rate=0.003),
    validation_split=0.05,
    batch_size=64,
    validation_batch_size=65536,
    epochs=40,
    callbacks=[keras.callbacks.ReduceLROnPlateau(patience=3), keras.callbacks.EarlyStopping(patience=5)]
    )

    # 모델 학습
    model_instance.fit(X_train, y_train)

    # 예측
    valid_y_pred = model_instance.predict(X_valid)
    valid_y_proba = model_instance.predict_proba(X_valid)[:, 1]  # 양성 클래스의 확률만 저장

    # 평가
    metrics = evaluate_model(y_valid, valid_y_pred, valid_y_proba)
    metrics.update({
        'Model': 'keras-classifier',
        'Fold': fold,
    })
    
    results.append(metrics)
    probas.append(valid_y_proba)

# DataFrame 생성
results_df = pd.DataFrame(results)
results_df


Training model: 0it [00:00, ?it/s]

Epoch 1/40
1367/1367 ━━━━━━━━━━━━━━━━━━━━ 9s 4ms/step - loss: 0.2831 - val_loss: 0.2642 - learning_rate: 0.0030
Epoch 2/40
1367/1367 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 0.2702 - val_loss: 0.2645 - learning_rate: 0.0030
Epoch 3/40
1367/1367 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - loss: 0.2697 - val_loss: 0.2629 - learning_rate: 0.0030
Epoch 4/40
1367/1367 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - loss: 0.2675 - val_loss: 0.2639 - learning_rate: 0.0030
Epoch 5/40
1367/1367 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - loss: 0.2665 - val_loss: 0.2637 - learning_rate: 0.0030
Epoch 6/40
1367/1367 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - loss: 0.2690 - val_loss: 0.2638 - learning_rate: 0.0030
Epoch 7/40
1367/1367 ━━━━━━━━━━━━━━━━━━━━ 9s 5ms/step - loss: 0.2598 - val_loss: 0.2613 - learning_rate: 3.0000e-04
Epoch 8/40
1367/1367 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 0.2647 - val_loss: 0.2617 - learning_rate: 3.0000e-04
Epoch 9/40
1367/1367 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.2602 - val_loss: 0.2613 - learni

Training model: 1it [01:17, 77.62s/it]

Epoch 1/40
1367/1367 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 0.2829 - val_loss: 0.2703 - learning_rate: 0.0030
Epoch 2/40
1367/1367 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.2670 - val_loss: 0.2723 - learning_rate: 0.0030
Epoch 3/40
1367/1367 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 0.2677 - val_loss: 0.2692 - learning_rate: 0.0030
Epoch 4/40
1367/1367 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.2690 - val_loss: 0.2704 - learning_rate: 0.0030
Epoch 5/40
1367/1367 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.2671 - val_loss: 0.2738 - learning_rate: 0.0030
Epoch 6/40
1367/1367 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.2639 - val_loss: 0.2700 - learning_rate: 0.0030
Epoch 7/40
1367/1367 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 0.2633 - val_loss: 0.2687 - learning_rate: 3.0000e-04
Epoch 8/40
1367/1367 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 0.2620 - val_loss: 0.2690 - learning_rate: 3.0000e-04
Epoch 9/40
1367/1367 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.2632 - val_loss: 0.2693 - learni

Training model: 2it [02:14, 65.38s/it]

...

Epoch 1/40
1367/1367 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 0.2838 - val_loss: 0.2652 - learning_rate: 0.0030
Epoch 2/40
1367/1367 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.2677 - val_loss: 0.2657 - learning_rate: 0.0030
Epoch 3/40
1367/1367 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 0.2675 - val_loss: 0.2645 - learning_rate: 0.0030
Epoch 4/40
1367/1367 ━━━━━━━━━━━━━━━━━━━━ 7s 5ms/step - loss: 0.2685 - val_loss: 0.2668 - learning_rate: 0.0030
Epoch 5/40
1367/1367 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 0.2688 - val_loss: 0.2656 - learning_rate: 0.0030
Epoch 6/40
1367/1367 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 0.2660 - val_loss: 0.2654 - learning_rate: 0.0030
Epoch 7/40
1367/1367 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - loss: 0.2645 - val_loss: 0.2633 - learning_rate: 3.0000e-04
Epoch 8/40
1367/1367 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 0.2626 - val_loss: 0.2634 - learning_rate: 3.0000e-04
Epoch 9/40
1367/1367 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - loss: 0.2616 - val_loss: 0.2637 - learni

Training model: 4it [05:52, 89.46s/it] 

Epoch 1/40
1367/1367 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - loss: 0.2817 - val_loss: 0.2697 - learning_rate: 0.0030
Epoch 2/40
1367/1367 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.2744 - val_loss: 0.2684 - learning_rate: 0.0030
Epoch 3/40
1367/1367 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 0.2680 - val_loss: 0.2690 - learning_rate: 0.0030
Epoch 4/40
1367/1367 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.2671 - val_loss: 0.2688 - learning_rate: 0.0030
Epoch 5/40
1367/1367 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.2684 - val_loss: 0.2671 - learning_rate: 0.0030
Epoch 6/40
1367/1367 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.2672 - val_loss: 0.2669 - learning_rate: 0.0030
Epoch 7/40
1367/1367 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.2653 - val_loss: 0.2698 - learning_rate: 0.0030
Epoch 8/40
1367/1367 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - loss: 0.2655 - val_loss: 0.2682 - learning_rate: 0.0030
Epoch 9/40
1367/1367 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.2650 - val_loss: 0.2672 - learning_rate:

Training model: 5it [07:03, 84.64s/it]


,Accuracy,ROC AUC,Model,Fold
0,0.876749,0.855069,keras-classifier,0
1,0.877097,0.854273,keras-classifier,1
2,0.876923,0.855334,keras-classifier,2
3,0.877005,0.853496,keras-classifier,3
4,0.875962,0.853751,keras-classifier,4


In [40]:
# 비교검증할 추가모델 리스트
models = [
    ('CatBoost', CatBoostClassifier(random_state=42, verbose=0)),
    ('LightGBM', LGBMClassifier(random_state=42)),
    ('XGBoost', XGBClassifier(random_state=42))
]


# 모델 학습 및 평가
for name, model in models:
    fold_results = []
    print(f"Training model: {name}")
    for fold, (train_index, valid_index) in enumerate(tqdm(skf.split(X, y), desc=f"{name} CV")):
        X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
        y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]

        # 모델 학습
        model.fit(X_train, y_train)

        # 예측
        valid_y_pred = model.predict(X_valid)
        valid_y_proba = model.predict_proba(X_valid)[:, 1]  # 양성 클래스의 확률만 저장

        # 평가
        metrics = evaluate_model(y_valid, valid_y_pred, valid_y_proba)
        metrics.update({'Model': name, 'Fold': fold})
        fold_results.append(metrics)

    results.extend(fold_results)

# DataFrame 생성
results_df = pd.DataFrame(results)


Training model: CatBoost


CatBoost CV: 5it [02:25, 29.19s/it]


Training model: LightGBM


LightGBM CV: 0it [00:00, ?it/s]

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 11313, number of negative: 80725
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005798 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 798
[LightGBM] [Info] Number of data points in the train set: 92038, number of used features: 122
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.122917 -> initscore=-1.965096
[LightGBM] [Info] Start training from score -1.965096


...

LightGBM CV: 4it [00:07,  1.75s/it]

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 11313, number of negative: 80726
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004710 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 802
[LightGBM] [Info] Number of data points in the train set: 92039, number of used features: 124
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.122915 -> initscore=-1.965108
[LightGBM] [Info] Start training from score -1.965108


LightGBM CV: 5it [00:10,  2.06s/it]


Training model: XGBoost


XGBoost CV: 5it [00:23,  4.68s/it]


In [41]:
# 모델별 평균 Accuracy와 평균 ROC AUC 계산
summary_df = results_df.groupby('Model')[['Accuracy', 'ROC AUC']].mean().reset_index()

# 요약 데이터프레임 출력
print(summary_df)

              Model  Accuracy   ROC AUC
0          CatBoost  0.877547  0.866438
1          LightGBM  0.877851  0.865248
2           XGBoost  0.877555  0.864283
3  keras-classifier  0.876747  0.854385


개선된 버전...

In [4]:
# 데이터 샘플링
train = train_origin.sample(frac=0.01, random_state = 42)

# 예측에 필요 없는 'id'와 'Annual_Premium' 변수를 드롭
train = train.drop(columns=['id'])

# 범주형 변수(2~3개 클래스) 인코딩
def encoding(train):
    gender_mapping = {'Male': 0, 'Female': 1}
    vehicle_age_mapping = {'< 1 Year': 0, '1-2 Year': 1, '> 2 Years': 2}
    vehicle_damage_mapping = {'No': 0, 'Yes': 1}

    train['Gender'] = train['Gender'].map(gender_mapping)
    train['Vehicle_Age'] = train['Vehicle_Age'].map(vehicle_age_mapping)
    train['Vehicle_Damage'] = train['Vehicle_Damage'].map(vehicle_damage_mapping)

    return train

train = encoding(train)

'''(같은 건데 왠지 위 방식이 미세하게 잘 나온다)
cat_columns_simple = ['Gender','Vehicle_Age','Vehicle_Damage']
train[cat_columns_simple] = train[cat_columns_simple].astype('category')
train['Gender'] = train['Gender'].cat.codes
train['Vehicle_Age'] = train['Vehicle_Age'].cat.codes
train['Vehicle_Damage'] = train['Vehicle_Damage'].cat.codes
'''

#Previously_Insured 교차항
'''
train['Previously_Insured_Annual_Premium'] = pd.factorize(train['Previously_Insured'].astype(str) + train['Annual_Premium'].astype(str))[0]
train['Previously_Insured_Vehicle_Age'] = pd.factorize(train['Previously_Insured'].astype(str) + train['Vehicle_Age'].astype(str))[0]
train['Previously_Insured_Vehicle_Damage'] = pd.factorize(train['Previously_Insured'].astype(str) + train['Vehicle_Damage'].astype(str))[0]
train['Previously_Insured_Vintage'] = pd.factorize(train['Previously_Insured'].astype(str) + train['Vintage'].astype(str))[0]
'''

#Age 교차항
train['Age_bins'] = pd.cut(train['Age'], bins=7).cat.codes
train['Age_x_Vehicle_Age'] = train['Age_bins'] * train['Vehicle_Age']
train['Age_x_Vehicle_Damage'] = train['Age_bins'] * train['Vehicle_Damage']
train['Age_x_Previously_Insured'] = train['Age_bins'] * train['Previously_Insured']

# 범주형 변수 타겟 인코딩
cat_columns = ['Region_Code', 'Policy_Sales_Channel', 'Vintage']
train.loc[:,cat_columns] = train.loc[:,cat_columns].astype('category')

target_encoder = TargetEncoder()
train[cat_columns] = target_encoder.fit_transform(train[cat_columns],train['Response'])

#수치형 변수 + 타겟 인코딩 변수 표준화
scaler = MinMaxScaler()
num_columns = ['Age', 'Annual_Premium','Region_Code', 'Policy_Sales_Channel', 'Vintage']
train[num_columns] = scaler.fit_transform(train[num_columns])

# 예측변수 분리 및 train, valid set 분리
X = train.drop(['Response'], axis=1)
y = train['Response']

In [7]:
# 모델 정의 함수
def get_model2(meta):
    model = keras.models.Sequential()
    model.add(keras.layers.Input(meta["X_shape_"][1:]))
    model.add(keras.layers.Dense(64, kernel_initializer='he_normal', activation='relu'))
    model.add(keras.layers.BatchNormalization()),
    model.add(keras.layers.Dense(128, kernel_initializer='he_normal', activation='relu'))
    model.add(keras.layers.BatchNormalization()),
    model.add(keras.layers.Dense(256, kernel_initializer='he_normal', activation='relu'))
    model.add(keras.layers.BatchNormalization()),
    model.add(keras.layers.Dense(128, kernel_initializer='he_normal', activation='relu'))
    model.add(keras.layers.BatchNormalization()),
    model.add(keras.layers.Dense(32, kernel_initializer='he_normal', activation='relu'))
    model.add(keras.layers.Dense(1, kernel_initializer='he_normal', activation='sigmoid'))
    
    return model

# 학습률 스케줄러 함수 정의
def lr_scheduler(epoch, lr):
    decay_rate = 0.96
    decay_step = 10
    new_lr = lr * (decay_rate ** (epoch // decay_step))
    print(f"Epoch {epoch}: Learning rate is {new_lr}")
    return new_lr

# LearningRateScheduler 콜백 설정
lr_schedule = LearningRateScheduler(lr_scheduler)

In [11]:
# 교차 검증 설정
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
X = train.drop(columns=['Response'])
y = train['Response']

# 결과를 저장할 리스트 초기화
results = []
probas = []

# 평가 함수 정의
def evaluate_model(y_true, y_pred, y_proba):
    return {
        'Accuracy': accuracy_score(y_true, y_pred),
        'ROC AUC': roc_auc_score(y_true, y_proba)
    }


# 모델 학습 및 평가
for fold, (train_index, valid_index) in enumerate(tqdm(skf.split(X, y), desc="Training model")):
    X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]

    # 모델 초기화 (새로운 인스턴스를 생성)
    model_instance = KerasClassifier(
    get_model2,
    loss="binary_crossentropy",
    optimizer=keras.optimizers.AdamW(learning_rate=2e-03),
    validation_split=0.05,
    batch_size=128,
    validation_batch_size=65536,
    epochs=30,  # Increase the number of epochs for further training
    callbacks=[lr_schedule, keras.callbacks.EarlyStopping(patience=20)]
    )

    # 모델 학습
    model_instance.fit(X_train, y_train)

    # 예측
    valid_y_pred = model_instance.predict(X_valid)
    valid_y_proba = model_instance.predict_proba(X_valid)[:, 1]  # 양성 클래스의 확률만 저장

    # 평가
    metrics = evaluate_model(y_valid, valid_y_pred, valid_y_proba)
    metrics.update({
        'Model': 'keras-classifier',
        'Fold': fold,
    })
    
    results.append(metrics)
    probas.append(valid_y_proba)

# DataFrame 생성
results_df = pd.DataFrame(results)
results_df

Training model: 0it [00:00, ?it/s]

Epoch 0: Learning rate is 0.0020000000949949026
Epoch 1/100
684/684 ━━━━━━━━━━━━━━━━━━━━ 23s 13ms/step - loss: 0.2815 - val_loss: 0.2655 - learning_rate: 0.0020
Epoch 1: Learning rate is 0.0020000000949949026
Epoch 2/100
684/684 ━━━━━━━━━━━━━━━━━━━━ 11s 16ms/step - loss: 0.2629 - val_loss: 0.2593 - learning_rate: 0.0020
Epoch 2: Learning rate is 0.0020000000949949026
Epoch 3/100
684/684 ━━━━━━━━━━━━━━━━━━━━ 11s 16ms/step - loss: 0.2658 - val_loss: 0.2580 - learning_rate: 0.0020
Epoch 3: Learning rate is 0.0020000000949949026
Epoch 4/100
684/684 ━━━━━━━━━━━━━━━━━━━━ 18s 12ms/step - loss: 0.2596 - val_loss: 0.2568 - learning_rate: 0.0020
Epoch 4: Learning rate is 0.0020000000949949026
Epoch 5/100
684/684 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step - loss: 0.2611 - val_loss: 0.2640 - learning_rate: 0.0020
Epoch 5: Learning rate is 0.0020000000949949026
Epoch 6/100
684/684 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - loss: 0.2618 - val_loss: 0.2577 - learning_rate: 0.0020
Epoch 6: Learning rate is 0.00200000

Training model: 1it [06:51, 411.23s/it]

Epoch 0: Learning rate is 0.0020000000949949026
Epoch 1/100
684/684 ━━━━━━━━━━━━━━━━━━━━ 12s 9ms/step - loss: 0.2829 - val_loss: 0.2683 - learning_rate: 0.0020
Epoch 1: Learning rate is 0.0020000000949949026
Epoch 2/100
684/684 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - loss: 0.2650 - val_loss: 0.2676 - learning_rate: 0.0020
Epoch 2: Learning rate is 0.0020000000949949026
Epoch 3/100
684/684 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - loss: 0.2628 - val_loss: 0.2652 - learning_rate: 0.0020
Epoch 3: Learning rate is 0.0020000000949949026
Epoch 4/100
684/684 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - loss: 0.2612 - val_loss: 0.2649 - learning_rate: 0.0020
Epoch 4: Learning rate is 0.0020000000949949026
Epoch 5/100
684/684 ━━━━━━━━━━━━━━━━━━━━ 6s 9ms/step - loss: 0.2614 - val_loss: 0.2642 - learning_rate: 0.0020
Epoch 5: Learning rate is 0.0020000000949949026
Epoch 6/100
684/684 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - loss: 0.2621 - val_loss: 0.2644 - learning_rate: 0.0020
Epoch 6: Learning rate is 0.0020000000949949

Training model: 2it [13:22, 399.29s/it]

...

Epoch 0: Learning rate is 0.0020000000949949026
Epoch 1/100
684/684 ━━━━━━━━━━━━━━━━━━━━ 14s 10ms/step - loss: 0.2847 - val_loss: 0.2612 - learning_rate: 0.0020
Epoch 1: Learning rate is 0.0020000000949949026
Epoch 2/100
684/684 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - loss: 0.2618 - val_loss: 0.2572 - learning_rate: 0.0020
Epoch 2: Learning rate is 0.0020000000949949026
Epoch 3/100
684/684 ━━━━━━━━━━━━━━━━━━━━ 7s 9ms/step - loss: 0.2621 - val_loss: 0.2584 - learning_rate: 0.0020
Epoch 3: Learning rate is 0.0020000000949949026
Epoch 4/100
684/684 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - loss: 0.2636 - val_loss: 0.2617 - learning_rate: 0.0020
Epoch 4: Learning rate is 0.0020000000949949026
Epoch 5/100
684/684 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - loss: 0.2632 - val_loss: 0.2584 - learning_rate: 0.0020
Epoch 5: Learning rate is 0.0020000000949949026
Epoch 6/100
684/684 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - loss: 0.2632 - val_loss: 0.2591 - learning_rate: 0.0020
Epoch 6: Learning rate is 0.0020000000949949

Training model: 4it [23:18, 328.51s/it]

Epoch 0: Learning rate is 0.0020000000949949026
Epoch 1/100
684/684 ━━━━━━━━━━━━━━━━━━━━ 15s 12ms/step - loss: 0.2830 - val_loss: 0.2607 - learning_rate: 0.0020
Epoch 1: Learning rate is 0.0020000000949949026
Epoch 2/100
684/684 ━━━━━━━━━━━━━━━━━━━━ 11s 12ms/step - loss: 0.2653 - val_loss: 0.2656 - learning_rate: 0.0020
Epoch 2: Learning rate is 0.0020000000949949026
Epoch 3/100
684/684 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - loss: 0.2626 - val_loss: 0.2623 - learning_rate: 0.0020
Epoch 3: Learning rate is 0.0020000000949949026
Epoch 4/100
684/684 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step - loss: 0.2618 - val_loss: 0.2591 - learning_rate: 0.0020
Epoch 4: Learning rate is 0.0020000000949949026
Epoch 5/100
684/684 ━━━━━━━━━━━━━━━━━━━━ 10s 7ms/step - loss: 0.2618 - val_loss: 0.2594 - learning_rate: 0.0020
Epoch 5: Learning rate is 0.0020000000949949026
Epoch 6/100
684/684 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step - loss: 0.2620 - val_loss: 0.2622 - learning_rate: 0.0020
Epoch 6: Learning rate is 0.0020000000949

Training model: 5it [28:31, 342.28s/it]


,Accuracy,ROC AUC,Model,Fold
0,0.876749,0.866675,keras-classifier,0
1,0.877662,0.868914,keras-classifier,1
2,0.879487,0.870949,keras-classifier,2
3,0.877961,0.864625,keras-classifier,3
4,0.878439,0.866306,keras-classifier,4


In [15]:
results_df['ROC AUC'].mean()

0.8674938487265186